This notebook will be mainly used for the capstone project.

In [58]:
!conda install -c conda-forge folium=0.5.0 --yes

Solving environment: ...working... failed with initial frozen solve. Retrying with flexible solve.
Solving environment: ...working... done

## Package Plan ##

  environment location: C:\Users\boike\Anaconda3

  added / updated specs:
    - folium=0.5.0


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    altair-4.1.0               |             py_1         614 KB  conda-forge
    branca-0.4.1               |             py_0          26 KB  conda-forge
    certifi-2019.9.11          |           py37_0         147 KB  conda-forge
    folium-0.5.0               |             py_0          45 KB  conda-forge
    vincent-0.4.4              |             py_1          28 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         860 KB

The following NEW packages will be INSTALLED:

  altair             conda-forge

In [55]:
!conda install -c conda-forge geopy --yes

Solving environment: ...working... done

## Package Plan ##

  environment location: C:\Users\boike\Anaconda3

  added / updated specs:
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    conda-4.8.3                |   py37hc8dfbb8_1         3.1 MB  conda-forge
    geographiclib-1.50         |             py_0          34 KB  conda-forge
    geopy-2.0.0                |     pyh9f0ad1d_0          63 KB  conda-forge
    python_abi-3.7             |          1_cp37m           4 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         3.2 MB

The following NEW packages will be INSTALLED:

  geographiclib      conda-forge/noarch::geographiclib-1.50-py_0
  geopy              conda-forge/noarch::geopy-2.0.0-pyh9f0ad1d_0
  python_abi         conda-forge/win-64::python_abi-3.7-1_cp37m

The following packag



==> WARNING: A newer version of conda exists. <==
  current version: 4.8.2
  latest version: 4.8.3

Please update conda by running

    $ conda update -n base -c defaults conda




In [56]:
conda update -n base -c defaults conda

Solving environment: ...working... done

## Package Plan ##

  environment location: C:\Users\boike\Anaconda3

  added / updated specs:
    - conda


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    backports.functools_lru_cache-1.6.1|             py_0          11 KB
    conda-4.8.3                |           py37_0         2.8 MB
    conda-package-handling-1.6.1|   py37h62dcd97_0         612 KB
    future-0.18.2              |           py37_1         646 KB
    ------------------------------------------------------------
                                           Total:         4.1 MB

The following packages will be REMOVED:

  python_abi-3.7-1_cp37m

The following packages will be UPDATED:

  backports.functoo~                               1.5-py_2 --> 1.6.1-py_0
  conda-package-han~                   1.6.0-py37h62dcd97_0 --> 1.6.1-py37h62dcd97_0
  future                              

In [59]:
import pandas as pd
import numpy as np
import requests
import json # library to handle JSON files

from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

import folium # map rendering library

In [6]:
print("Hello Capstone Project Course!")

Hello Capstone Project Course!


<h2>scraping the wikipedia table using BeautifulSoup</h2>

In [13]:
website_url = requests.get("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M").text

In [60]:
from bs4 import BeautifulSoup
soup = BeautifulSoup(website_url,'lxml')
#print(soup.prettify())

In [61]:
My_table = soup.find('table',{'class':'wikitable sortable'})

<h2>Initializing a dataframe, and parsing through the data scrape to populate the rows</h2>

In [101]:
column_names = ['PostalCode', 'Borough', 'Neighborhood']

In [102]:
# instantiate the dataframe
neighborhoods = pd.DataFrame(columns=column_names)

In [103]:
count = 1
postal_code = ''
borough = ''
neighborhood = ''
for item in My_table.findAll('td'):
    for c in item:
        if count == 1:
            postal_code = c.replace('\n','')
            count += 1
        elif count == 2:
            borough = c.replace('\n','')
            count += 1
        elif count == 3:
            neighborhood = c.replace('\n','')
            count = 1
            neighborhoods = neighborhoods.append({'PostalCode': postal_code,
                                          'Borough': borough,
                                          'Neighborhood': neighborhood},
                                          ignore_index=True)

In [104]:
neighborhoods.head()

,PostalCode,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


In [108]:
#drop records where the borough = 'Not assigned'
neighborhoods.drop(neighborhoods[neighborhoods['Borough'] == 'Not assigned'].index, inplace = True)

In [109]:
neighborhoods.head()

,PostalCode,Borough,Neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


In [110]:
#make sure there are no neighborhoods with the value of 'not assigned'. if there are, assign them the name of the borough
neighborhoods['Neighborhood'] = np.where(neighborhoods['Neighborhood'] == 'Not assigned', neighborhoods['Borough'],neighborhoods['Neighborhood'])

In [111]:
neighborhoods.head()

,PostalCode,Borough,Neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


In [112]:
#verify there are no more neighborhoods with 'not assigned' values
neighborhoods[neighborhoods['Neighborhood'].str.contains("assigned")]

,PostalCode,Borough,Neighborhood


In [114]:
#check to make sure that there are no duplicate postal codes that need to be condensed into one line
neighborhoods[neighborhoods['PostalCode'].duplicated() == True]

,PostalCode,Borough,Neighborhood
